In [28]:
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

METADATA = {'Contributor': 'T. Dunn'}
SAVEFIGS = False

In [29]:
# Setup template and html config for plotly figures.
from bs4 import BeautifulSoup
import plotly.graph_objects as go

TFB_TEMPLATE = dict(
    layout=go.Layout(
        template='plotly_white',
        autosize=False,
        margin=dict(l=60, r=25, b=55, t=20, pad=4),
        font=dict(family='sans-serif', size=14),
        hovermode='closest',
        xaxis=dict(
            automargin=False,
            showline=True,
            linecolor='#444',
            linewidth=2,
            mirror=True,
            ticks='outside',
            minor=dict(tickmode='linear', ticks='outside'),
            zeroline=False,
        ),
        yaxis=dict(
            automargin=False,
            showline=True,
            linecolor='#444',
            linewidth=2,
            mirror=True,
            ticks='outside',
            minor=dict(tickmode='linear', ticks='outside'),
            zeroline=False,
        ),
        hoverlabel=dict(align='left'),
    )
)

TFB_CONFIG = {
    'responsive': True, # must be True to auto-scale when resizing
    'autosizable': True, # doesn't impact auto rescaling
    'showAxisDragHandles': False,
    'displaylogo': False,
    'displayModeBar': 'hover',
    'modeBarButtonsToRemove': [
        'select2d',
        'lasso2d',
        'zoom2d',
        'zoomIn2d',
        'zoomOut2d',
        'pan2d',
        'autoScale2d',
        'hoverClosestCartesian',
        'hoverCompareCartesian',
        'toggleSpikelines',
        'resetScale2d',
    ],
    'toImageButtonOptions': {
        'format': 'png', # one of png, svg, jpeg, webp
        'filename': 'tfb-plot',
        'height': 450,
        'width': 600,
        'scale': 2
    },
}

def save_fig_html(fig, format, name, **kwargs):
    file_name = f'./{name}.html'

    match format:
        case 'large':
            fig.update_traces(line_width=2, marker_size=8)
            fig.update_shapes(line_width=2)
            fig.update_layout(font_size=14)
            fig.update_annotations(font_size=14)
        case 'small':
            fig.update_traces(line_width=1, marker_size=6)
            fig.update_shapes(line_width=1)
            fig.update_layout(font_size=10, legend=dict(font=dict(size=10)))
            fig.update_annotations(font_size=10)

    fig_html = fig.to_html(
        config=TFB_CONFIG,
        include_plotlyjs=False, 
        full_html=False, 
    )
    fig_soup = BeautifulSoup(fig_html, 'html.parser')
    fig_soup.div['class'] = f'plotly-div-{format}'
    if kwargs.get('style', None):
        fig_soup.div['style'] = kwargs.get('style')
    with open(file_name, 'wb') as fout:
        fout.write(fig_soup.prettify('utf-8'))

In [30]:
def effective_HP_linear(row):
    return row['HP Mean']*(1 + (0.05/0.6)*(row['AC'] - 13))/np.sqrt(0.6)

def effective_DPR_linear(row):
    return row['DPR Mean']*(1 + (0.05/0.6)*(row['AB'] - 3))*np.sqrt(0.6)

def effective_XP_linear(row):
    return 0.25*row['HP Mean']*row['DPR Mean']*(1 + (0.05/0.6)*(row['AC'] + row['AB'] - 16))

def rounds_to_win(row, diff):
    return 3.0 * np.sqrt(row[diff + ' XP']/row['Medium XP'])

In [31]:
# plots the ratio of a typical monster's effective DPR and effective HP as a function of CR.
dfD0 = pd.read_csv('../../assets/data/dmg-targets.csv') # 'CR','XP','HP Mean','AC','DPR Mean','AB'
dfD0['eHP'] = dfD0.apply(lambda row: effective_HP_linear(row), axis=1)
dfD0['eDPR'] = dfD0.apply(lambda row: effective_DPR_linear(row), axis=1)
dfD0['eXP'] = dfD0.apply(lambda row: effective_XP_linear(row), axis=1)

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=TFB_TEMPLATE,
        xaxis=dict(
            title_text='challenge rating',
            range=[-0.5,30.5],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='eDPR / eHP',
            range=[0.0,0.4],
            tick0=0, dtick=0.1,
            minor=dict(tick0=0, dtick=0.05),
        ),
    )
)

# plot data
fig.add_trace(go.Scatter(
    x=dfD0['CR'], 
    y=dfD0['eDPR']/dfD0['eHP'],
    mode='markers+lines', 
    marker_size=6,
    hovertemplate= 
            '<b>CR %{x}</b><br>' +
            'eDPR/eHP %{y:,.2f}' + 
            '<extra></extra>'
))

# show figure
fig.update_layout(width=600, height=500)
fig.show(config=TFB_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    save_fig_html(fig, format='large', name='fig-monster-edpr-ehp-ratio-by-cr-large')
    save_fig_html(fig, format='small', name='fig-monster-edpr-ehp-ratio-by-cr-small')

In [32]:
# plots the number of rounds combat is expected to take for encounters at each difficulty threshold as a function of PC level
dfP0 = pd.read_csv('../../assets/data/pc-thresholds.csv') # 'Level','Easy XP','Medium XP','Hard XP','Deadly XP','Daily XP'
dfP0['Easy Rounds'] = dfP0.apply(lambda row: rounds_to_win(row, 'Easy'), axis=1)
dfP0['Medium Rounds'] = dfP0.apply(lambda row: rounds_to_win(row, 'Medium'), axis=1)
dfP0['Hard Rounds'] = dfP0.apply(lambda row: rounds_to_win(row, 'Hard'), axis=1)
dfP0['Deadly Rounds'] = dfP0.apply(lambda row: rounds_to_win(row, 'Deadly'), axis=1)


# create figure
fig = go.Figure(
    layout=go.Layout(
        template=TFB_TEMPLATE,
        xaxis=dict(
            title_text='level',
            range=[0,21],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='rounds',
            range=[0,5],
            tick0=0, dtick=1,
            minor=dict(tick0=0, dtick=0.5),
        ),
        legend=dict(
            xanchor='left', yanchor='bottom',
            x=0.00, y=0.00,
            orientation='h',
            tracegroupgap=0,
        )
    )
)

# plot data
for d in ['Easy', 'Medium', 'Hard', 'Deadly']:
    #ax.plot(dfP0['Level'], dfP0['{} Rounds'.format(d)], label=d)
    fig.add_trace(go.Scatter(
        x=dfP0['Level'], 
        y=dfP0[f'{d} Rounds'],
        mode='markers+lines', 
        #line=dict(color=color),
        name=d,
        hovertemplate= 
            f'<b>{d}</b><br>'
            'Level %{x}<br>'+
            'Rounds %{y:,.1f}' + 
            '<extra></extra>'
    ))

# show figure
fig.update_layout(width=600, height=500)
fig.show(config=TFB_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    save_fig_html(fig, format='large', name='fig-rounds-per-encounter-by-level-large')
    save_fig_html(fig, format='small', name='fig-rounds-per-encounter-by-level-small')

In [18]:
# plots the number of rounds combat is expected to take for encounters at each difficulty threshold
dfP0 = pd.read_csv('../../assets/data/pc-thresholds.csv') # 'Level','Easy XP','Medium XP','Hard XP','Deadly XP','Daily XP'
dfP0['Easy Rounds'] = dfP0.apply(lambda row: rounds_to_win(row, 'Easy'), axis=1)
dfP0['Medium Rounds'] = dfP0.apply(lambda row: rounds_to_win(row, 'Medium'), axis=1)
dfP0['Hard Rounds'] = dfP0.apply(lambda row: rounds_to_win(row, 'Hard'), axis=1)
dfP0['Deadly Rounds'] = dfP0.apply(lambda row: rounds_to_win(row, 'Deadly'), axis=1)
dfPG = dfP0.mean()

diffs = ['Easy', 'Medium', 'Hard', 'Deadly']
rounds = [dfPG['Easy Rounds'], dfPG['Medium Rounds'], dfPG['Hard Rounds'], dfPG['Deadly Rounds']]

# save table
df1 = pd.DataFrame(data={'Difficulty': diffs, 'Rounds': rounds})
df1.to_html('./table-rounds-per-encounter.html', index=False, classes='center', float_format='{:,.2f}'.format, border=0)


# create figure
fig = go.Figure(
    layout=go.Layout(
        template=TFB_TEMPLATE,
        xaxis=dict(
            title_text='encounter difficulty',
        ),
        yaxis=dict(
            title_text='rounds',
            range=[0,5],
            tick0=0, dtick=1,
            minor=dict(tick0=0, dtick=0.5),
        ),
    )
)

# plot results
fig.add_trace(go.Scatter(
    x=diffs, 
    y=rounds,
    mode='markers+lines', 
    marker_size=10,
    hovertemplate= 
            '<b>%{x}</b><br>' +
            'Rounds %{y:,.1f}' + 
            '<extra></extra>'
))


# show figure
fig.update_layout(width=600, height=500)
fig.show(config=TFB_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    save_fig_html(fig, format='large', name='fig-rounds-per-encounter-large')
    save_fig_html(fig, format='small', name='fig-rounds-per-encounter-small')

In [35]:
# plots the average number of encounters per full adventuring day for each difficulty threshold as a function of PC level
dfP0 = pd.read_csv('../../assets/data/pc-thresholds.csv') # 'Level','Easy XP','Medium XP','Hard XP','Deadly XP','Daily XP'
dfP0['Easy Encounters']   = dfP0['Daily XP']/dfP0['Easy XP']
dfP0['Medium Encounters'] = dfP0['Daily XP']/dfP0['Medium XP']
dfP0['Hard Encounters']   = dfP0['Daily XP']/dfP0['Hard XP']
dfP0['Deadly Encounters'] = dfP0['Daily XP']/dfP0['Deadly XP']

diffs = ['Easy', 'Medium', 'Hard', 'Deadly']


# create figure
fig = go.Figure(
    layout=go.Layout(
        template=TFB_TEMPLATE,
        xaxis=dict(
            title_text='level',
            range=[0,21],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='encounters per day',
            range=[0,21],
            tick0=0, dtick=2,
            minor=dict(tick0=0, dtick=1),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='h',
            tracegroupgap=0,
        )
    )
)

# plot data
for d in diffs:
    #ax.plot(dfP0['Level'], dfP0['{} Encounters'.format(d)], label=d)
    fig.add_trace(go.Scatter(
        x=dfP0['Level'], 
        y=dfP0[f'{d} Encounters'],
        mode='markers+lines', 
        #line=dict(color=color),
        name=d,
        hovertemplate= 
            f'<b>{d}</b><br>'
            'Level %{x}<br>'+
            'Encounters %{y:,.1f}' + 
            '<extra></extra>'
    ))


# show figure
fig.update_layout(width=600, height=500)
fig.show(config=TFB_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    save_fig_html(fig, format='large', name='fig-encounters-per-full-adventuring-day-by-level-large')
    save_fig_html(fig, format='small', name='fig-encounters-per-full-adventuring-day-by-level-small')

In [17]:
# plots the average number of encounters per full adventuring day for each difficulty threshold
dfP0 = pd.read_csv('../../assets/data/pc-thresholds.csv') # 'Level','Easy XP','Medium XP','Hard XP','Deadly XP','Daily XP'
dfP0['Easy Encounters']   = dfP0['Daily XP']/dfP0['Easy XP']
dfP0['Medium Encounters'] = dfP0['Daily XP']/dfP0['Medium XP']
dfP0['Hard Encounters']   = dfP0['Daily XP']/dfP0['Hard XP']
dfP0['Deadly Encounters'] = dfP0['Daily XP']/dfP0['Deadly XP']

dfPG = dfP0.mean()

diffs = ['Easy', 'Medium', 'Hard', 'Deadly']
rounds = [dfPG['Easy Encounters'], dfPG['Medium Encounters'], dfPG['Hard Encounters'], dfPG['Deadly Encounters']]


# create figure
fig = go.Figure(
    layout=go.Layout(
        template=TFB_TEMPLATE,
        xaxis=dict(
            title_text='encounter difficulty',
        ),
        yaxis=dict(
            title_text='encounters per day',
            range=[0,14],
            tick0=0, dtick=2,
            minor=dict(tick0=0, dtick=1),
        ),
    )
)

# plot data
fig.add_trace(go.Scatter(
    x=diffs, 
    y=rounds,
    mode='markers+lines', 
    marker_size=10,
    hovertemplate= 
            '<b>%{x}</b><br>' +
            'Encounters %{y:,.1f}' + 
            '<extra></extra>'
))


# show figure
fig.update_layout(width=600, height=500)
fig.show(config=TFB_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    save_fig_html(fig, format='large', name='fig-encounters-per-full-adventuring-day-large')
    save_fig_html(fig, format='small', name='fig-encounters-per-full-adventuring-day-small')

In [34]:
# plots the average number of rounds per full adventuring day for each difficulty threshold as a function of PC level.
dfP0 = pd.read_csv('../../assets/data/pc-thresholds.csv') # 'Level','Easy XP','Medium XP','Hard XP','Deadly XP','Daily XP'
dfP0['Easy Encounters']   = dfP0['Daily XP']/dfP0['Easy XP']
dfP0['Medium Encounters'] = dfP0['Daily XP']/dfP0['Medium XP']
dfP0['Hard Encounters']   = dfP0['Daily XP']/dfP0['Hard XP']
dfP0['Deadly Encounters'] = dfP0['Daily XP']/dfP0['Deadly XP']

dfP0['Easy Rounds']   = dfP0.apply(lambda row: rounds_to_win(row, 'Easy'), axis=1)
dfP0['Medium Rounds'] = dfP0.apply(lambda row: rounds_to_win(row, 'Medium'), axis=1)
dfP0['Hard Rounds']   = dfP0.apply(lambda row: rounds_to_win(row, 'Hard'), axis=1)
dfP0['Deadly Rounds'] = dfP0.apply(lambda row: rounds_to_win(row, 'Deadly'), axis=1)

dfP0['Easy Rounds per Day']   = dfP0['Easy Encounters']   * dfP0['Easy Rounds']
dfP0['Medium Rounds per Day'] = dfP0['Medium Encounters'] * dfP0['Medium Rounds']
dfP0['Hard Rounds per Day']   = dfP0['Hard Encounters']   * dfP0['Hard Rounds']
dfP0['Deadly Rounds per Day'] = dfP0['Deadly Encounters'] * dfP0['Deadly Rounds']


# create figure
fig = go.Figure(
    layout=go.Layout(
        template=TFB_TEMPLATE,
        xaxis=dict(
            title_text='level',
            range=[0,21],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='rounds per day',
            range=[0,35],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        legend=dict(
            xanchor='left', yanchor='bottom',
            x=0.00, y=0.00,
            orientation='h',
            tracegroupgap=0,
        )
    )
)

# plot results
diffs = ['Easy', 'Medium', 'Hard', 'Deadly']
for d in diffs:
    fig.add_trace(go.Scatter(
        x=dfP0['Level'], 
        y=dfP0[f'{d} Rounds per Day'],
        mode='markers+lines', 
        #line=dict(color=color),
        name=d,
        hovertemplate= 
            f'<b>{d}</b><br>'
            'Level %{x}<br>'+
            'Rounds %{y:,.1f}' + 
            '<extra></extra>'
    ))


# save table
dfPG = dfP0.mean()
df1 = pd.DataFrame(data={
    'Difficulty': ['Easy', 'Medium', 'Hard', 'Deadly'], 
    'Rounds per Encounter': [dfPG['Easy Rounds'], dfPG['Medium Rounds'], dfPG['Hard Rounds'], dfPG['Deadly Rounds']],
    'Encounters per Day': [dfPG['Easy Encounters'], dfPG['Medium Encounters'], dfPG['Hard Encounters'], dfPG['Deadly Encounters']],
    'Rounds per Day': [dfPG['Easy Rounds per Day'], dfPG['Medium Rounds per Day'], dfPG['Hard Rounds per Day'], dfPG['Deadly Rounds per Day']]
})
df1.to_html('./table-rounds-summary.html', index=False, classes='center', float_format='{:,.1f}'.format, border=0, table_id='tab:rounds-summary')


# show figure
fig.update_layout(width=600, height=500)
fig.show(config=TFB_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    save_fig_html(fig, format='large', name='fig-rounds-per-full-adventuring-day-by-level-large')
    save_fig_html(fig, format='small', name='fig-rounds-per-full-adventuring-day-by-level-small')

In [16]:
# plots the average number of rounds per full adventuring day for each difficulty threshold
dfP0 = pd.read_csv('../../assets/data/pc-thresholds.csv') # 'Level','Easy XP','Medium XP','Hard XP','Deadly XP','Daily XP'
dfP0['Easy Encounters']   = dfP0['Daily XP']/dfP0['Easy XP']
dfP0['Medium Encounters'] = dfP0['Daily XP']/dfP0['Medium XP']
dfP0['Hard Encounters']   = dfP0['Daily XP']/dfP0['Hard XP']
dfP0['Deadly Encounters'] = dfP0['Daily XP']/dfP0['Deadly XP']

dfP0['Easy Rounds']   = dfP0.apply(lambda row: rounds_to_win(row, 'Easy'), axis=1)
dfP0['Medium Rounds'] = dfP0.apply(lambda row: rounds_to_win(row, 'Medium'), axis=1)
dfP0['Hard Rounds']   = dfP0.apply(lambda row: rounds_to_win(row, 'Hard'), axis=1)
dfP0['Deadly Rounds'] = dfP0.apply(lambda row: rounds_to_win(row, 'Deadly'), axis=1)

dfP0['Easy Rounds per Day']   = dfP0['Easy Encounters']   * dfP0['Easy Rounds']
dfP0['Medium Rounds per Day'] = dfP0['Medium Encounters'] * dfP0['Medium Rounds']
dfP0['Hard Rounds per Day']   = dfP0['Hard Encounters']   * dfP0['Hard Rounds']
dfP0['Deadly Rounds per Day'] = dfP0['Deadly Encounters'] * dfP0['Deadly Rounds']

dfPG = dfP0.mean()

diffs = ['Easy', 'Medium', 'Hard', 'Deadly']
rounds = [dfPG['Easy Rounds per Day'], dfPG['Medium Rounds per Day'], dfPG['Hard Rounds per Day'], dfPG['Deadly Rounds per Day']]


# create figure
fig = go.Figure(
    layout=go.Layout(
        template=TFB_TEMPLATE,
        xaxis=dict(
            title_text='encounter difficulty'
        ),
        yaxis=dict(
            title_text='rounds per day',
            range=[0,30],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        )
    )
)

fig.add_trace(go.Scatter(
    x=diffs, 
    y=rounds,
    mode='markers+lines', 
    marker_size=10,
    hovertemplate= 
            '<b>%{x}</b><br>' +
            'Rounds %{y:,.1f}' + 
            '<extra></extra>'
))

# show figure
fig.update_layout(width=600, height=500)
fig.show(config=TFB_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    save_fig_html(fig, format='large', name='fig-rounds-per-full-adventuring-day-large')
    save_fig_html(fig, format='small', name='fig-rounds-per-full-adventuring-day-small')